# data

In [1]:
%load_ext autoreload
%autoreload 2
import importlib
from importlib import reload  
  
import monotonic
import utils
from train import execute_model_evaluation
from train_config import FlatACLConfig
import train_config
import data_manager
from data_manager import DatasetManager
import metrics
from dataclasses import replace

reload(utils)
reload(monotonic)
reload(train_config)
%load_ext autoreload
%autoreload 2

data_mgr = DatasetManager(device='cuda')
# data_mgr.load_csv('weather', './weather.csv')
data_mgr.load_csv('weather', './weather.csv', random_feature_num_seed_pair=(10, 1955))
# Select Random Features for 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Original Num Columns: 21, Selected Num Columns: 10
Selected Columns: ['wv (m/s)' 'max. wv (m/s)' 'PAR (�mol/m�/s)' 'VPdef (mbar)'
 'H2OC (mmol/mol)' 'Tlog (degC)' 'Tdew (degC)' 'rain (mm)' 'rho (g/m**3)'
 'VPact (mbar)']

Dataset: weather (csv)
Shape: torch.Size([52616, 10])
Channels: 10
Length: 52616
Source: ./weather.csv

Sample data (first 2 rows):
tensor([[ 1.0200e+00,  1.6000e+00,  0.0000e+00,  8.9000e-01,  5.4900e+00,
          1.1450e+01, -1.3300e+00,  0.0000e+00,  1.2806e+03,  5.5400e+00],
        [ 4.3000e-01,  8.4000e-01,  0.0000e+00,  9.5000e-01,  5.4500e+00,
          1.1510e+01, -1.4400e+00,  0.0000e+00,  1.2803e+03,  5.4900e+00]])


# Exp - 336

## Seq=336 : Normal

### EigenACL

#### pred=96

In [40]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
    

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
global_std.shape: torch.Size([10])
Global Std for weather: tensor([0.9609, 0.9487, 0.9034, 0.9395, 0.9149, 0.9772, 0.9213, 1.0041, 0.9573,
        0.9151], device='cuda:0')
Train set sample shapes: torch.Size([336, 10]), torch.Size([96, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([96, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 285
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 96, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 285
Validation Batches: 38
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.3956, mae: 0.3616, hub

#### pred=196

In [41]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
    

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
global_std.shape: torch.Size([10])
Global Std for weather: tensor([0.9609, 0.9487, 0.9034, 0.9395, 0.9149, 0.9772, 0.9213, 1.0041, 0.9573,
        0.9151], device='cuda:0')
Train set sample shapes: torch.Size([336, 10]), torch.Size([196, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([196, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 284
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 196, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 284
Validation Batches: 37
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4561, mae: 0.4091,

#### pred=336

##### model I

In [4]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
    

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
global_std.shape: torch.Size([10])
Global Std for weather: tensor([0.9609, 0.9487, 0.9034, 0.9395, 0.9149, 0.9772, 0.9213, 1.0041, 0.9573,
        0.9151], device='cuda:0')
Train set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5047, mae: 0.4441,

##### model II 

In [36]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=True,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=True,
    

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4857, mae: 0.4337, huber: 0.1745, swd: 0.2616, target_std: 0.8450
Epoch [1/50], Val Losses: mse: 0.6642, mae: 0.4158, huber: 0.1696, swd: 0.4187, target_std: 0.6777
Epoch [1/50], Test Losses:

##### model III

In [37]:
from matplotlib.font_manager import font_family_aliases
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 4,
    householder_reflects_data = 8,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
    

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4975, mae: 0.4394, huber: 0.1785, swd: 0.2810, target_std: 0.8450
Epoch [1/50], Val Losses: mse: 0.6716, mae: 0.4227, huber: 0.1725, swd: 0.4086, target_std: 0.6777
Epoch [1/50], Test Losses:

#### pred=720

In [42]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['weather']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only',
    # single_magnitude_for_shift=True,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False,
    

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU,
                                              nn.LogSigmoid]
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
global_std.shape: torch.Size([10])
Global Std for weather: tensor([0.9609, 0.9487, 0.9034, 0.9395, 0.9149, 0.9772, 0.9213, 1.0041, 0.9573,
        0.9151], device='cuda:0')
Train set sample shapes: torch.Size([336, 10]), torch.Size([720, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([720, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 280
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 720, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 280
Validation Batches: 33
Test Batches: 74

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5550, mae: 0.4763,

### Timemixer

#### pred=96

In [30]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([96, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([96, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 285
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 96, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 285
Validation Batches: 38
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4042, mae: 0.3580, huber: 0.1366, swd: 0.2109, target_std: 0.8448
Epoch [1/50], Val Losses: mse: 0.5257, mae: 0.3093, huber: 0.1166,

#### pred=196

In [29]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([196, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([196, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 284
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 196, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 284
Validation Batches: 37
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4540, mae: 0.3994, huber: 0.1591, swd: 0.2165, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.6064, mae: 0.3605, huber: 0.1

#### pred=336

In [2]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
global_std.shape: torch.Size([10])
Global Std for weather: tensor([0.9609, 0.9487, 0.9034, 0.9395, 0.9149, 0.9772, 0.9213, 1.0041, 0.9573,
        0.9151], device='cuda:0')
Train set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==========================================

#### pred=720

In [27]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([720, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([720, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 280
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 720, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 280
Validation Batches: 33
Test Batches: 74

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5846, mae: 0.4825, huber: 0.2120, swd: 0.2400, target_std: 0.8457
Epoch [1/50], Val Losses: mse: 0.7746, mae: 0.4612, huber: 0.2

### PatchTST

#### pred=96

In [26]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([96, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([96, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 285
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 96, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 285
Validation Batches: 38
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4311, mae: 0.3839, huber: 0.1504, swd: 0.2042, target_std: 0.8447
Epoch [1/50], Val Losses: mse: 0.5569, mae: 0.3473, huber: 0.1325,

#### pred=196

In [25]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([196, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([196, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 284
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 196, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 284
Validation Batches: 37
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4904, mae: 0.4260, huber: 0.1746, swd: 0.2141, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.6292, mae: 0.3782, huber: 0.1

#### 336-336

In [24]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5453, mae: 0.4627, huber: 0.1974, swd: 0.2250, target_std: 0.8450
Epoch [1/50], Val Losses: mse: 0.7011, mae: 0.4248, huber: 0.1

#### pred=720

In [23]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['weather']['channels'],
    enc_in=data_mgr.datasets['weather']['channels'],
    dec_in=data_mgr.datasets['weather']['channels'],
    c_out=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([720, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([720, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 280
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 720, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 280
Validation Batches: 33
Test Batches: 74

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.6158, mae: 0.5042, huber: 0.2254, swd: 0.2344, target_std: 0.8456
Epoch [1/50], Val Losses: mse: 0.8006, mae: 0.4850, huber: 0.2

### DLinear

#### 336-96

In [22]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([96, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([96, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 285
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 96, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 285
Validation Batches: 38
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.3897, mae: 0.3567, huber: 0.1330, swd: 0.2161, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.5212, mae: 0.3090, huber: 0.1138,

#### 336-196

In [21]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([196, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([196, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 284
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 196, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 284
Validation Batches: 37
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4454, mae: 0.3998, huber: 0.1565, swd: 0.2310, target_std: 0.8449
Epoch [1/50], Val Losses: mse: 0.5819, mae: 0.3531, huber: 0.1

#### 336-336

In [20]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([336, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 336, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.4938, mae: 0.4350, huber: 0.1770, swd: 0.2461, target_std: 0.8450
Epoch [1/50], Val Losses: mse: 0.6391, mae: 0.4002, huber: 0.1

#### 336-720

In [19]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['weather']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('weather', cfg, data_mgr, scale=True)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36831, 10])
Shape of validation data: torch.Size([5261, 10])
Shape of testing data: torch.Size([10524, 10])
Train set sample shapes: torch.Size([336, 10]), torch.Size([720, 10])
Validation set sample shapes: torch.Size([336, 10]), torch.Size([720, 10])
Test set data shapes: torch.Size([10524, 10]), torch.Size([10524, 10])
Number of batches in train_loader: 280
Batch 0: Data shape torch.Size([128, 336, 10]), Target shape torch.Size([128, 720, 10])

Data Preparation: weather
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: Yes
Train Split: 0.7
Val Split: 0.8
Training Batches: 280
Validation Batches: 33
Test Batches: 74

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 0.5517, mae: 0.4720, huber: 0.2001, swd: 0.2662, target_std: 0.8457
Epoch [1/50], Val Losses: mse: 0.7203, mae: 0.4432, huber: 0.1